In [3]:
import sys
import os
sys.path.append("../../src/")
sys.path.append("../../utilities/")
import dataset_class
from dataset_class import dataset
import data_utils
import evaluation_utils
import numpy as np
import pandas as pd
import seaborn as sns

In [41]:
from importlib import reload
evaluation_utils = reload(evaluation_utils)

In [12]:
signals_dir = '../../data/GM12878/hg19/genomic-assays/bin_100000/'
TAD_path = '../../data/GM12878/hg19/annotations/GSE63525_GM12878_primary+replicate_Arrowhead_domainlist.txt'
SC_path = '../../data/GM12878/hg19/annotations/GSE63525_GM12878_subcompartments.bed'
CTCF_bedpe_path = '../../data/GM12878/hg19/ChIA-PET/CTCF/lifted_data'
RNAPII_bedpe_path = '../../data/GM12878/hg19/ChIA-PET/RNAPII/lifted_data'
gene_expression_path = '../../data/GM12878/hg19/gene_expression/GM12878_genes_RPKM.txt'
RT_path = '../../data/GM12878/hg19/RT/six_phase/six_phases.txt'
TAD_agreements = {}
CTCF_ChIA_PETs = {}
RNAPII_ChIA_PETs = {}
resolution = 100000
annotation_dir = '../../data/GM12878/hg19/res100000_datasets/annotations/K=8/'

# Average length

In [54]:
avg_lengths = []
methods = ['GMM', 'HMM_map', 'HMM_viterbi', 'kmeans']
inputs = ['functional', 'structural', 'combined']
for method in methods:
    for input in inputs:
        annotation_path = os.path.join(annotation_dir, '{}_{}_annotation.txt'.format(method, input))
        df = pd.read_csv(annotation_path, header = None, sep = "\t")
        df['length'] = df.iloc[:,2] - df.iloc[:,1]
        avg_length = np.mean(df['length'])
        avg_lengths.append({'method': method, 'input': input, 'avg_length': avg_length})
avg_lengths = pd.DataFrame(avg_lengths)

In [58]:
avg_lengths2 = []
methods = ['short_gbr', 'long_gbr', 'short_hmrf', 'long_hmrf']
for method in methods:
    for i in range(10):
        annotation_path = os.path.join(annotation_dir, method, '{}_{}_annotation.txt'.format(method,i))
        df = pd.read_csv(annotation_path, header = None, sep = "\t")
        df['length'] = df.iloc[:,2] - df.iloc[:,1]
        avg_length = np.mean(df['length'])
        avg_lengths2.append({'method': method, 'iter': i, 'avg_length': avg_length})
avg_lengths2 = pd.DataFrame(avg_lengths2)

# Number of domains

In [15]:
num_domains = []
methods = ['GMM', 'HMM_map', 'HMM_viterbi', 'kmeans']
inputs = ['functional', 'structural', 'combined']
for method in methods:
    for input in inputs:
        annotation_path = os.path.join(annotation_dir, '{}_{}_annotation.txt'.format(method, input))
        num_domain = pd.read_csv(annotation_path, header = None).shape[0]
        num_domains.append({'method': method, 'input': input, 'num_domain': num_domain})
num_domains = pd.DataFrame(num_domains)

In [17]:
num_domains2 = []
methods = ['short_gbr', 'long_gbr', 'short_hmrf', 'long_hmrf']
for method in methods:
    for i in range(10):
        annotation_path = os.path.join(annotation_dir, method, '{}_{}_annotation.txt'.format(method,i))
        num_domain = pd.read_csv(annotation_path, header = None).shape[0]
        num_domains2.append({'method': method, 'iter': i, 'num_domain': num_domain})
num_domains2 = pd.DataFrame(num_domains2)

# Gene expression

In [4]:
gene_expression_ve = []
methods = ['GMM', 'HMM_map', 'HMM_viterbi', 'kmeans']
inputs = ['functional', 'structural', 'combined']
for method in methods:
    for input in inputs:
        annotation_path = os.path.join(annotation_dir, '{}_{}_annotation.txt'.format(method, input))
        ge_ve = evaluation_utils.gene_expression_ve(gene_expression_path, annotation_path, resolution)
        gene_expression_ve.append({'method': method, 'input': input, 'ge_ve': ge_ve})
gene_expression_ve = pd.DataFrame(gene_expression_ve)

In [7]:
gene_expression_ve2 = []
methods = ['short_gbr', 'long_gbr', 'short_hmrf', 'long_hmrf']
for method in methods:
    for i in range(10):
        annotation_path = os.path.join(annotation_dir, method, '{}_{}_annotation.txt'.format(method,i))
        ge_ve = evaluation_utils.gene_expression_ve(gene_expression_path, annotation_path, resolution)
        gene_expression_ve2.append({'method': method, 'iter': i, 'ge_ve': ge_ve})
gene_expression_ve2 = pd.DataFrame(gene_expression_ve2)

In [13]:
SC_ge_ve = evaluation_utils.gene_expression_ve(gene_expression_path, SC_path, resolution)

# TAD agreement

In [45]:
TAD_agreements = []
methods = ['GMM', 'HMM_map', 'HMM_viterbi', 'kmeans']
inputs = ['functional', 'structural', 'combined']
for method in methods:
    for input in inputs:
        annotation_path = os.path.join(annotation_dir, '{}_{}_annotation.txt'.format(method, input))
        TAD_agreement = evaluation_utils.normalized_TAD_agreement(annotation_path, TAD_path, resolution, 20)
        TAD_agreements.append({'method': method, 'input': input, 'TAD_agreement': TAD_agreement})
TAD_agreements = pd.DataFrame(TAD_agreements)

In [47]:
TAD_agreements2 = []
methods = ['short_gbr', 'long_gbr', 'short_hmrf', 'long_hmrf']
for method in methods:
    for i in range(10):
        annotation_path = os.path.join(annotation_dir, method, '{}_{}_annotation.txt'.format(method,i))
        TAD_agreement = evaluation_utils.normalized_TAD_agreement(annotation_path, TAD_path, resolution, 20)
        TAD_agreements2.append({'method': method, 'iter': i, 'TAD_agreement': TAD_agreement})
TAD_agreements2 = pd.DataFrame(TAD_agreements2)

# Replication timing scores

In [49]:
RT_scores = []
methods = ['GMM', 'HMM_map', 'HMM_viterbi', 'kmeans']
inputs = ['functional', 'structural', 'combined']
for method in methods:
    for input in inputs:
        annotation_path = os.path.join(annotation_dir, '{}_{}_annotation.txt'.format(method, input))
        RT_score1, RT_score2 = evaluation_utils.RT_scores(RT_path, annotation_path, resolution)
        RT_scores.append({'method': method, 'input': input, 'RT_score1': RT_score1, 'RT_score2': RT_score2})
RT_scores = pd.DataFrame(RT_scores)

In [60]:
RT_scores2 = []
methods = ['short_gbr', 'long_gbr', 'short_hmrf', 'long_hmrf']
for method in methods:
    for i in range(10):
        annotation_path = os.path.join(annotation_dir, method, '{}_{}_annotation.txt'.format(method,i))
        RT_score1, RT_score2 = evaluation_utils.RT_scores(RT_path, annotation_path, resolution)
        RT_scores2.append({'method': method, 'iter': i, 'RT_score1': RT_score1, 'RT_score2': RT_score2})
RT_scores2 = pd.DataFrame(RT_scores2)

# long-range-interactions

In [186]:
CTCF_OE = evaluation_utils.OE_intra_of_bedpe(CTCF_bedpe_path, annotation_path, resolution)

In [188]:
RNAPII_OE = evaluation_utils.OE_intra_of_bedpe(RNAPII_bedpe_path, annotation_path, resolution)

# save

In [62]:
stats = pd.merge(avg_lengths, num_domains, on = ['method', 'input'])
stats = pd.merge(stats, gene_expression_ve, on = ['method', 'input'])
stats = pd.merge(stats, TAD_agreements, on = ['method', 'input'])
stats = pd.merge(stats, RT_scores, on = ['method', 'input'])

In [64]:
stats.to_csv('results/res100000/stats.txt', sep = '\t', index = False)

In [67]:
stats2 = pd.merge(avg_lengths2, num_domains2, on = ['method', 'iter'])
stats2 = pd.merge(stats2, gene_expression_ve2, on = ['method', 'iter'])
stats2 = pd.merge(stats2, TAD_agreements2, on = ['method', 'iter'])
stats2 = pd.merge(stats2, RT_scores2, on = ['method', 'iter'])

In [68]:
stats2.to_csv('results/res100000/stats2.txt', sep = '\t', index = False)